In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("pharmacy_claims.csv.gz")
print(df.shape)
df.head()

/var/folders/nm/vy6jh_gd4fv2xcmvqc69gc_80000gn/T/ipykernel_37210/331727412.py:1: DtypeWarning: Columns (14,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("pharmacy_claims.csv.gz")


(2570133, 48)


,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c38,_c39,_c40,_c41,_c42,_c43,_c44,_c45,_c46,_c47
0,351051,99K1,NaN,NaN,GH06,99K1E001,IMD Non-Medigap,RISK,BlueChoice,Maryland,...,30,90.0,0,5,0.00,0.00,0.0,0.0,0.0,0.00
1,351051,99K1,NaN,NaN,GH06,99K1E001,IMD Non-Medigap,RISK,BlueChoice,Maryland,...,-28,-2.0,0,5,-8608.84,-4080.68,-150.0,0.0,-150.0,-3780.68
2,351051,99K1,NaN,NaN,GH06,99K1E001,IMD Non-Medigap,RISK,BlueChoice,Maryland,...,28,6.0,0,0,0.00,0.00,0.0,0.0,0.0,0.00
3,351051,99K1,NaN,NaN,GH06,99K1E001,IMD Non-Medigap,RISK,BlueChoice,Maryland,...,28,6.0,0,0,0.00,0.00,0.0,0.0,0.0,0.00
4,351051,99K1,NaN,NaN,GH06,99K1E001,IMD Non-Medigap,RISK,BlueChoice,Maryland,...,28,6.0,0,0,0.00,0.00,0.0,0.0,0.0,0.00


In [3]:
data_dict = pd.read_excel('Claims/Pharmacy_Claims/Data Dictionary Pharmacy Claims.xlsx', skiprows=1)
data_dict.head()

,Field,Definition
0,Member Life ID,Unique Identifier
1,Group Number,The groups are defined based on the account's ...
2,Section,The groups are defined based on the account's ...
3,Department,The groups are defined based on the account's ...
4,Package Code / Class Code,The groups are defined based on the account's ...


In [4]:
col_names = data_dict['Field'].apply(lambda x: (x.lower().split("/")[0].split("(")[0].strip().replace(" ", "_"))).tolist()
df.columns = col_names

In [5]:
df = df[df['member_life_id']!=-1]
df.shape

(1512707, 48)

In [6]:
df = df[df["subgroup"]!='001901088MD3000008K6GW']

In [7]:
df["unique_key"] = df["claim_number"].astype(str)+ "_" + \
    "_" + df["paid_date"].astype(str)+ "_" + df["paid_amount"].astype(str)+ "_" + df["billed_amount"].astype(str)

In [8]:
print(df.shape)
print(df['unique_key'].nunique())

(1494236, 49)
1494236


In [9]:
cols_to_gen = [
    'member_life_id', 'birth_date', 'gender_code',
    'subscriber_zip', 'claim_number', 'prescription_number',
    'ndc', 'drug_name', 'specialty_drug_indicator', 'generic_indicator',
    'drug_tier', 'fill_date', 'paid_date', 'days_supply', 'quantity_dispensed',
    'refill_number', 'number_of_refills', 'billed_amount', 'paid_amount', 'unique_key'
]
df = df[cols_to_gen]

In [10]:
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()

In [11]:
metadata.detect_from_dataframe(data=df)

In [12]:
python_dict = metadata.to_dict()
python_dict

{'columns': {'member_life_id': {'sdtype': 'numerical'},
  'birth_date': {'sdtype': 'categorical'},
  'gender_code': {'sdtype': 'categorical'},
  'subscriber_zip': {'sdtype': 'categorical'},
  'claim_number': {'sdtype': 'numerical'},
  'prescription_number': {'sdtype': 'numerical'},
  'ndc': {'sdtype': 'numerical'},
  'drug_name': {'sdtype': 'categorical'},
  'specialty_drug_indicator': {'sdtype': 'categorical'},
  'generic_indicator': {'sdtype': 'categorical'},
  'drug_tier': {'sdtype': 'categorical'},
  'fill_date': {'sdtype': 'categorical'},
  'paid_date': {'sdtype': 'categorical'},
  'days_supply': {'sdtype': 'numerical'},
  'quantity_dispensed': {'sdtype': 'numerical'},
  'refill_number': {'sdtype': 'numerical'},
  'number_of_refills': {'sdtype': 'numerical'},
  'billed_amount': {'sdtype': 'numerical'},
  'paid_amount': {'sdtype': 'numerical'},
  'unique_key': {'sdtype': 'categorical'}},
 'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1'}

In [13]:
metadata.update_column(
    column_name='unique_key',
    sdtype='id')
metadata.update_column(
    column_name='member_life_id',
    sdtype='categorical')
metadata.update_column(
    column_name='claim_number',
    sdtype='categorical')
metadata.update_column(
    column_name='prescription_number',
    sdtype='categorical')
# metadata.update_column(
#     column_name='relationship_code',
#     sdtype='categorical')
# metadata.update_column(
#     column_name='claim_disposition_code',
#     sdtype='categorical')
metadata.update_column(
    column_name='ndc',
    sdtype='categorical')
metadata.update_column(
    column_name='fill_date',
    sdtype='datetime',
    datetime_format='%Y-%m-%d')
metadata.update_column(
    column_name='paid_date',
    sdtype='datetime',
    datetime_format='%Y-%m-%d')
metadata.update_column(
    column_name='birth_date',
    sdtype='datetime',
    datetime_format='%Y-%m-%d')
metadata.set_primary_key(column_name='unique_key')
# metadata.add_alternate_keys(column_names=['claim_number', 'pharmacy_number', 'prescription_number'])

In [14]:
from sdv.single_table import GaussianCopulaSynthesizer

synthesizer = GaussianCopulaSynthesizer(metadata)
synthesizer.fit(df)

synthetic_data = synthesizer.sample(num_rows=100000)

In [15]:
synthetic_data.head()

,member_life_id,birth_date,gender_code,subscriber_zip,claim_number,prescription_number,ndc,drug_name,specialty_drug_indicator,generic_indicator,drug_tier,fill_date,paid_date,days_supply,quantity_dispensed,refill_number,number_of_refills,billed_amount,paid_amount,unique_key
0,34472558,1973-07-17,F,20854,183404148698104997,797774,64720013210,VALSART/HCTZ TAB 160-25MG,N,Generic drug,PREF BR NM DRGS HIGHER CPY,2017-07-28,2017-09-03,16,816.306,7,1,-2348.83,124.62,sdv-id-0
1,26144317,2004-06-29,F,20910,170943515085183999,6322658,904054480,PAIN RELIEF CAP 325MG,N,Generic drug,GENERIC DRUGS LOWEST CPY,2017-09-24,2017-07-11,31,1719.021,2,0,-650.05,56.25,sdv-id-1
2,28033327,1950-06-09,F,20837,183200421011162999,1510029,93742698,LISINOPRIL TAB 20MG,N,Brand drug,Other,2018-06-12,2018-04-22,67,593.214,5,0,862.90,-9.62,sdv-id-2
3,2656542,2006-04-03,F,21754,172986920943133999,7020435,169750111,NORETHINDRON TAB 0.35MG,N,Brand drug,GENERIC DRUGS LOWEST CPY,2017-03-08,2017-01-31,69,115.930,1,0,1161.08,93.20,sdv-id-3
4,3026646,1969-09-12,M,22066,171100342917187999,281207513,64764091830,VYVANSE CAP 30MG,N,Brand drug,GENERIC DRUGS LOWEST CPY,2018-04-26,2018-06-11,-44,-1401.593,7,0,1778.72,51.06,sdv-id-4


In [16]:
synthetic_data.nunique()

member_life_id               30700
birth_date                   25984
gender_code                      2
subscriber_zip                5495
claim_number                 96189
prescription_number          88397
ndc                          10705
drug_name                     4021
specialty_drug_indicator         2
generic_indicator                4
drug_tier                        8
fill_date                      730
paid_date                     1002
days_supply                    314
quantity_dispensed           98804
refill_number                   40
number_of_refills                8
billed_amount                93136
paid_amount                  34605
unique_key                  100000
dtype: int64

In [19]:
from sdv.evaluation.single_table import run_diagnostic

diagnostic_report = run_diagnostic(
    real_data=df,
    synthetic_data=synthetic_data,
    metadata=metadata)

Generating report ...


  0%|                                                    | 0/20 [00:00<?, ?it/s]

(1/3) Evaluating Coverage: :   0%|                       | 0/20 [00:00<?, ?it/s]

(1/3) Evaluating Coverage: :  15%|██▎            | 3/20 [00:00<00:01, 16.21it/s]

(1/3) Evaluating Coverage: :  25%|███▊           | 5/20 [00:00<00:02,  5.31it/s]

(1/3) Evaluating Coverage: :  30%|████▌          | 6/20 [00:01<00:02,  5.20it/s]

(1/3) Evaluating Coverage: :  40%|██████         | 8/20 [00:01<00:01,  6.51it/s]

(1/3) Evaluating Coverage: :  45%|██████▊        | 9/20 [00:01<00:01,  6.88it/s]

(1/3) Evaluating Coverage: :  50%|███████       | 10/20 [00:01<00:01,  6.91it/s]

(1/3) Evaluating Coverage: : 100%|██████████████| 20/20 [00:01<00:00, 11.53it/s]


  0%|                                                    | 0/20 [00:00<?, ?it/s]

(2/3) Evaluating Boundary: : 100%|█████████████| 20/20 [00:00<00:00, 238.68it/s]


  0%|                                                     | 0/1 [00:00<?